# Embodied Scan
## 研究背景与动机
* 传统3D感知研究多基于全局视角的场景级输入，缺乏第一视角，连续感知的支持，Embodied Scan<br>
1. 从第一视角观察中理解3D场景<br>
2. 将视觉信息与语言指令相结合<br>
* 现有数据集规模小，标注有限，输入不匹配

## Embodied Scan数据集
* 数据规模和构成<br>
1. 5185个场景，包含来自ScanNet的1513个场景<br>
2. 近百万个第一视角RGB-D图像，覆盖多样化的室内环境<br>
3. 160000个带方向的3D边界框
4. 超过760个物体类别
5. 100万个语言提示，描述物体位置和属性
6. 密集语义占据标注，涵盖80个常见物体类别
* 数据来源与处理<br>
整合了多个现有数据集，包括ScanNet、Matterport3D和3RScan，通过统一的坐标系、关键帧采样、区域划分等方式，将多源数据整合为一致的格式。
* 标注方法
1. 3D边界框标注：使用SAM辅助标注工具，结合人工校正，确保边界框的准确性和一致性。
2. 语义占据占据：基于点云分割标注，体素化为40*40*160的体素网格。
3. 基于SR3D方法生成，描述物体间的空间关系（如“椅子在桌子旁边”）
* 统计特性
1. 开放词汇标注，覆盖日常生活中的多种物体
2. 超过288个类别拥有10个以上的实例
3. 语言提示数量是现有数据集SR3D的10倍以上

## 数据集
#### 数据来源选择
1. 核心目标：重新利用现有3D场景数据集，生成大规模的第一视角RGB-D图像和丰富的标注信息
2. 数据来源：<br>
ScanNet：包含密集的RGB-D视频序列的相机姿态，适合做连续，时序感知任务<br>
Matterport3D：供高质量、全景的建筑级3D扫描，视角更广，适合模拟多视图感知<br>
3RScan：同样提供时序扫描，且包含场景变化，增加了数据的多样性<br>
3. 为什么这么选<br>
**(1)它们都提供RGB-D图像和相机姿态信息，适合生成第一视角数据**<br>
**(2)它们互补：ScanNet和3RScan强调“视频流”，Matterport3D强调“空间覆盖”**<br>
**(3)关键改造：传统上，这些数据集主要用于场景级的任务（输入是整个重建好的点云/网格）。EmbodiedScan将它们 “解构” 为一系列第一人称的、连续的观测流，模拟机器人在环境中探索时的真实输入**<br>
#### 数据处理与对齐
1. 全局坐标系对齐<br>
- **问题**：不同数据集的坐标系不一致，直接使用会导致空间位置混乱<br>
- **解决方案**：遵循ScanNet的约定，为每个场景建立一个统一的全局坐标系。原点在场景中心附近，XY平面与地板对齐，坐标轴与墙面方向对齐<br>
- **重要性**：这是多视图融合的基础。所有在不同时间、不同视角拍摄的RGB-D帧，其像素/点云都可以通过相机位姿转换到这个全局坐标系下进行叠加和计算，从而逐步构建出完整的3D场景表示
2. 帧选择与场景划分<br>
- **对ScanNet/3RScan（视频流）**：进行关键帧采样（如每10帧取1帧），在保证信息连续性的同时减少冗余，提高处理效率<br>
- **对Matterport3D（全景图）**：将其划分成更小的区域（如单个房间）。因为原始场景太大（整栋楼），一个智能体通常只在一个区域内活动。然后选取那些视角中心落在该区域内的图像<br>
- **统一输入接口**：最终，模型被设计成可以处理两种输入：时序连续的帧序列 和 无序的多视图图像集合，这覆盖了真实场景中可能遇到的不同情况
#### 多模态标注体系（核心贡献）
1. 3D带方向边界框标注<br>
- **目的**：为场景中的每个物体提供精确的3D位置和朝向信息（对于机器人抓取，导航避障至关重要）<br>
- **方法**：采用 “SAM辅助 + 人工精修” 的半自动化流程：<br>
    - 自动初筛：在选定的关键帧上，运行强大的 Segment Anything Model，生成大量2D物体掩码和轴对齐框<br>
    - 人工精修平台：标注员在一个定制化的3D标注平台（基于SUSTech Points）上工作。平台同时显示：三个正交的3D视图（俯视、侧视等）&&2D的RGB和深度图像<br>
    - 高效标注：标注员在2D图像中点击SAM给出的候选掩码，平台会自动在3D空间中生成一个初始的3D立方体。标注员随后可以在3D视图中拖拽、旋转、缩放这个立方体，使其与点云中的物体精确对齐<br>
- **优势**：
- 解决了小物体和方向缺失问题：传统室内数据集（如ScanNet）的标注多为轴对齐或没有朝向，且小物体很少。此方法能高效标注大量小物体和任意朝向的物体<br>
- 质量高：最终经过人工检查，每个场景约需10-30分钟，确保了标注精度
2. 密集语义占据标注<br>
- **目的**：提供一个稠密的、结构化的场景几何与语义表示。与稀疏的边界框不同，占据栅格能描述任意形状的物体（如沙发、植物）和结构（如墙壁的凹凸），对机器人路径规划极其重要<br>
- **方法**：<br>
    - 点云分割：基于已有的点云语义分割结果（如ScanNet的20类标签），对场景点云进行语义分类<br>
    - 体素化：将分割后的点云体素化为40x40x160的体素网格。每个体素存储一个语义标签（对应80类常见物体）或为空<br>
3.  基于语言的空间关系标注<br>
- **目的**：捕捉物体之间的空间关系（如“椅子在桌子旁边”），这对理解场景布局和执行复杂任务（如“去拿桌子上的杯子”）非常重要<br>
- **方法**：<br>
    - 自动生成：基于SR3D的方法，利用3D边界框的位置关系，自动生成描述物体间空间关系的语言提示,关系类型包括：水平邻近、垂直邻近、支撑关系、以场景为参考的方位关系、介于关系<br>
    - 应对高密度挑战：EmbodiedScan物体密度很高，一个描述（如“椅子”）可能对应多个实例，导致指代模糊<br>
    - 解决方案：引入“关系链”概念，通过多步关系描述（如“椅子在桌子旁边，桌子在沙发前面”）来唯一定位目标物体<br>
- **规模**：生成了超过100万个语言提示，数量是现有数据集SR3D的10倍以上，且覆盖更丰富的关系类型
4. 数据集统计分析
- **词汇表构建**：<br>
    - 开放词汇标注：EmbodiedScan采用开放词汇标注策略，涵盖了日常生活中的多种物体类别，远超传统数据集的有限类别<br>
    - 后处理：使用Sentence-BERT将语义相似的名称聚类，并映射到WordNet节点，最后人工合并整理，形成了一个包含762个类别的词汇表。该词汇表与LVIS（550个）和COCO（50个室内类）有大量重叠<br>
- **实例统计**：<br>
    - 长尾分布：拥有超过760个类别，呈现典型的长尾分布。其中288个类别有超过10个实例，约400个类别超过5个实例<br>
    - 语言提示数量：超过100万个语言提示，远超现有数据集（如SR3D）的规模，支持更复杂的语言理解任务<br>
- **基准任务划分**：<br>
    - 3D检测：剔除了{墙，天花板，地板，杂物} 这些非目标类别，将剩余的284个类别按实例数量分为头类、普通类、尾类，用于评估模型在不同常见度物体上的性能<br>
    - 语义占据：选取了前80个最常见且对导航重要的类别（如空、地板、墙、椅子、门等）<br>
    - 语言提示理解：按场景划分训练/验证集，并包含大量复杂的、组合式的空间关系描述<br>

## Embodied  Perception基线模型框架
* 它是一个统一的、端到端的系统，旨在处理具身AI场景中复杂的多模态输入，并完成多种3D感知任务。其核心设计是：***一个模型，多种输入，多项任务***。它能够接受任意数量的第一视角RGB-D图像和语言提示，输出3D物体检测框、语义占据栅格以及基于语言的视觉定位结果。
#### 核心模块一：多模态3D编码器
* 这是模型的大脑，负责从原始输入中提取并融合信息。其设计目标是灵活与高效
1. 独立特征提取：<br>
- **2D图像编码器**：使用ResNet50 + FPN。输入N张RGB图像，输出多尺度（如4个层级）的2D特征图。这些特征富含物体的纹理、颜色和语义信息<br>
- **3D点云编码器**：深度图通过相机参数转换为点云，并转换到全局坐标系。使用Minkowski ResNet34（一种稀疏卷积网络）进行处理，输出多尺度的稀疏体素特征。这些特征精确编码了场景的几何结构<br>
- **语言编码器**：采用预训练的BERT模型，将输入的语言提示转换为上下文感知的文本特征向量，捕捉语义和关系信息<br>
2. 关键创新点一：可扩展的任意视图输入处理
- **3D点云处理**：将不同视角的点云统一变换到全局坐标系后简单拼接和下采样。由于3D坐标是绝对的，无论来自哪一帧，处理方式都一致<br>
- **2D图像特征融合**：将来自N个无序或有序视图的2D特征整合到3D空间中<br>
- 方法：对于3D空间中的一个查询点（如一个体素中心），通过透视投影找到它在每一张2D图像上的对应像素位置
- 聚合：然后，将这些对应位置的2D特征值进行平均池化，得到一个聚合后的2D特征
- 优势：这种操作是“排列不变的”，即无论输入图像的顺序如何，只要内容相同，聚合结果就一样。这使得模型可以处理任意数量、任意顺序的视图
3. 关键创新点二：跨模态特征融合
* 根据下游任务的特点，设计了不同的融合策略
- **稠密融合**：<br>
    - 目的：为语义占据任务提供丰富的多模态信息，该任务需要对整个3D空间进行密集、精细的分类<br>
    - 方法：<br>
        - 将2D图像特征通过FPN上采样到较高分辨率
        - 在预定义的密集3D网格（与真实标注的占据栅格同分辨率）的每个格点（体素）上，执行上述的“投影-平均池化”操作，将2D特征“涂抹”到整个3D空间，形成一个稠密的2D特征体积
        - 将这个2D特征体积与从点云提取的3D稀疏体素特征（通过插值或填充变为稠密）进行通道拼接
- **同构多级稀疏融合**：<br>
    - 目的：为3D检测和基于语言的视觉定位任务提供高效的多模态特征，这些任务只需关注场景中的关键物体位置<br>
    - 挑战：需要在多个尺度上融合2D和3D特征，同时保持稀疏表示以节省计算资源<br>
    - 方法：<br>
        - 利用同构编码器：图像ResNet和点云Minkowski ResNet都产出4个层级的特征，它们感受野逐级增大，语义信息逐级抽象
        - 层级对齐融合：让点云编码器第k层的稀疏特征，去查询图像编码器第k层的2D特征（同样通过投影和池化），然后将融合后的特征送入检测解码器
    - 优势：<br>
        - 稳定性：同层级的特征在抽象程度上是匹配的，融合更自然，训练更稳定
        - 丰富性：不同层级的特征融合了不同尺度（局部细节和全局上下文）的多模态信息，有利于检测不同大小的物体
- **视觉语言融合**：<br>
    - 目的：为基于语言的视觉定位任务，将语言信息与视觉特征结合，指导模型关注与指令相关的区域<br>
    - 方法：<br>
        - 使用跨模态注意力机制：将语言特征作为查询（Query），视觉特征（融合后的2D-3D特征）作为键（Key）和值（Value）
        - 通过注意力计算，生成与语言提示相关的视觉特征表示
    - 优势：这种机制允许模型动态地根据语言内容调整视觉关注点，提高定位准确性
#### 核心模块二：稀疏与稠密解码器
* 这是模型的执行器，负责将融合后的多模态特征转换为具体的任务输出。根据任务的不同，设计了两种解码器
1. 稀疏解码器：3D物体检测与基于语言的视觉定位
- **任务**：<br>
    - 3D物体检测：预测场景中每个物体的3D边界框及其类别<br>
    - 基于语言的视觉定位：根据语言提示，定位并预测对应物体的3D边界框<br>
- **结构**：基于3D稀疏卷积网络，处理同构多级融合后的稀疏特征
- **方法**：<br>
    - 3D检测头：使用基于VoteNet的检测头，包含投票模块和边界框回归模块，输出物体类别和位置<br>
    - 语言定位头：在检测头基础上，加入语言引导的注意力机制，将语言特征与视觉特征结合，输出与语言提示对应的物体边界框<br>
- **优势**：稀疏解码器高效处理大规模3D数据，适合检测和定位任务
- **核心思想**：将预测的8个角点与真实框的8个角点进行比较。为了更精准地监督，作者将回归目标分解为{中心，尺寸，朝向}三组。计算损失时，固定其中两组为真实值，只让模型学习另一组。例如，只让模型学习中心，而使用真实的尺寸和朝向与预测的中心组成一个“预测框”来计算角点损失。最后将三个损失加权求和。
- **目的**：让模型分别、清晰地学习位置、大小和朝向，避免了联合优化时的耦合与混淆。
2. 稠密解码器：语义占据预测
- **任务**：预测整个3D空间中每个体素的语义类别
- **结构**：基于3D卷积网络，处理稠密融合后的多模态特征体积
- **方法**：<br>
    - 使用一系列3D卷积层和上采样层，将融合特征体积逐步恢复到与真实占据栅格相同的分辨率
    - 最终输出每个体素的语义类别概率分布
3. 视觉定位解码器
- **任务**：根据语言提示，定位并预测对应物体的3D边界框
- **结构**：基于稀疏解码器，增加语言引导模块
- **方法**：<br>
    - 融合语言特征：使用跨模态注意力机制，将语言特征与视觉特征结合，生成与语言提示相关的视觉表示
    - 预测边界框：基于融合后的特征，使用类似3D检测头的方法，输出与语言提示对应的物体边界框